In [1]:
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import torch

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input

2022-02-02 12:52:49.526703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-02 12:52:49.526742: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
x_train = pd.read_csv('input_train__fppJZ22.csv')
y_train = pd.read_csv('y_train_CGZ7jXu.csv')

x_test = pd.read_csv('input_test__2scdXsn.csv')

y_rand = pd.read_csv('y_rand.csv')

In [3]:
x_train.groupby(['cluster']).size().sort_values().head(20)

cluster
707      20
1223     24
467      28
419      40
1327     40
68       44
733      55
948      66
975      66
1460     72
316      80
609      90
1017    105
150     105
1008    105
1000    105
149     105
1013    105
377     105
991     105
dtype: int64

In [4]:
#fill na values with the mean of the values in the cluster this day, then forward fill
for column in ['bc']+[f'ret_{i}' for i in range(1,24)]:
    print('filling '+column)
    x_train[column] = x_train.groupby(['day','cluster'])[column].transform(lambda x: x.fillna(x.mean()))
    x_test[column] = x_test.groupby(['day','cluster'])[column].transform(lambda x: x.fillna(x.mean()))
x_train = x_train.ffill(axis=1)
x_test = x_test.ffill(axis=1)

filling bc


KeyboardInterrupt: 

In [5]:
#fill na values of bc and ret_1 with the mean of the values in the cluster this day, then forward fill
for column in ['bc']+[f'ret_{i}' for i in range(1,2)]:
    print('filling '+column)
    x_train[column] = x_train.groupby(['day','cluster'])[column].transform(lambda x: x.fillna(x.mean()))
    x_test[column] = x_test.groupby(['day','cluster'])[column].transform(lambda x: x.fillna(x.mean()))
x_train = x_train.ffill(axis=1)
x_test = x_test.ffill(axis=1)

filling bc
filling ret_1


In [ ]:
x_test.isna().sum()

id         0
cluster    0
day        0
asset      0
md         0
bc         0
ret_1      0
ret_2      0
ret_3      0
ret_4      0
ret_5      0
ret_6      0
ret_7      0
ret_8      0
ret_9      0
ret_10     0
ret_11     0
ret_12     0
ret_13     0
ret_14     0
ret_15     0
ret_16     0
ret_17     0
ret_18     0
ret_19     0
ret_20     0
ret_21     0
ret_22     0
ret_23     0
dtype: int64

In [6]:
y_train['sample_id'] = x_train['cluster'] * 21 + x_test['day']

In [7]:
x_train['sample_id'] = x_train['cluster'] * 21 + x_train['day']
x_train = x_train.merge(y_train, on='sample_id')


In [8]:
y_train = x_train[['target', 'sample_id']]
y_train['target'] = (y_train['target'] +1)/2

/tmp/ipykernel_21284/1578065531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['target'] = (y_train['target'] +1)/2


In [9]:
y_train.set_index('sample_id', inplace=True)

# Essai 1 : dernier return

In [ ]:
y_test.isnull().values.sum()

0

In [ ]:
y_test.to_csv('y_test.csv')

# Essai 2 : DNN

In [ ]:
x_train.shape, y_train.shape

((491038, 31), (491038, 1))

In [ ]:
X = x_train.drop(columns=['id','asset','sample_id','target','day','cluster'])
y = y_train

,target
0,0.503452
1,0.503452
2,0.503452
3,0.503452
4,0.503452
...,...
491033,0.489653
491034,0.489653
491035,0.489653
491036,0.489653


In [ ]:
X['cluster'] = X['cluster']/1463

In [ ]:
inputs = Input(shape=(X.shape[1],))
x = tf.keras.layers.Dense(200, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(200, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
x_train[x_train['cluster']==0]

,id,cluster,day,asset,md,bc,ret_1,ret_2,ret_3,ret_4,...,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23,sample_id,target
0,0.0,0.0,17.0,10.0,-5.734573,0.125855,-0.002060,0.005848,-0.002736,-0.005487,...,0.001018,-0.002372,-0.000340,-0.000680,-0.007140,0.006849,0.002041,-0.001697,17.0,0.006903
1,5.0,0.0,17.0,12.0,-5.814399,0.500111,0.005916,0.013115,0.028039,-0.000565,...,0.009911,-0.006337,0.013601,-0.019710,-0.005793,0.003256,0.005979,0.006396,17.0,0.006903
2,34.0,0.0,17.0,13.0,-5.813696,0.418711,-0.022372,-0.003269,0.020044,0.011075,...,0.006606,0.014162,-0.009537,0.002407,-0.004117,-0.007578,0.008678,-0.007226,17.0,0.006903
3,64.0,0.0,17.0,0.0,-5.446527,0.860837,-0.010396,-0.001970,0.005921,0.005886,...,0.001291,0.020632,-0.000632,-0.000632,-0.007590,0.005099,0.005707,0.006305,17.0,0.006903
4,76.0,0.0,17.0,11.0,-5.799585,0.720519,-0.010064,0.014141,-0.001735,0.003791,...,0.008028,0.032467,-0.002909,0.002093,0.005695,0.002832,0.006797,-0.009909,17.0,0.006903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,299.0,0.0,1.0,6.0,-5.607835,0.685390,0.002309,0.004075,0.002294,-0.014965,...,0.029084,-0.012220,0.003870,-0.003971,-0.015539,-0.007148,0.003839,0.010219,1.0,-0.011966
395,312.0,0.0,1.0,3.0,-5.532724,0.683797,-0.002514,0.014493,-0.008696,-0.014411,...,0.021907,0.013241,0.023024,0.009124,-0.031344,-0.011823,0.008186,0.004372,1.0,-0.011966
396,341.0,0.0,1.0,2.0,-5.482508,0.570504,0.156307,0.156307,0.007247,-0.013008,...,0.005241,0.001890,0.001005,-0.000003,-0.008154,-0.000569,-0.000361,0.001555,1.0,-0.011966
397,355.0,0.0,1.0,5.0,-5.602152,0.631948,0.004337,-0.000864,0.006914,-0.012017,...,0.010563,0.010563,-0.005226,0.009632,-0.030356,-0.013417,0.007253,0.002700,1.0,-0.011966


In [ ]:
model.compile(optimizer='sgd', loss='mse')
model.fit(X, y, batch_size=21, epochs=10)

Epoch 1/10
23383/23383 [==============================] - 16s 659us/step - loss: 5.4744e-05
Epoch 2/10
23383/23383 [==============================] - 15s 660us/step - loss: 4.1930e-05
Epoch 3/10
23383/23383 [==============================] - 15s 660us/step - loss: 4.1556e-05
Epoch 4/10
23383/23383 [==============================] - 17s 716us/step - loss: 4.1298e-05
Epoch 5/10
23383/23383 [==============================] - 17s 733us/step - loss: 4.0927e-05
Epoch 6/10
23383/23383 [==============================] - 17s 727us/step - loss: 4.0631e-05
Epoch 7/10
23383/23383 [==============================] - 17s 729us/step - loss: 4.0502e-05
Epoch 8/10
23383/23383 [==============================] - 17s 727us/step - loss: 4.0415e-05
Epoch 9/10
23383/23383 [==============================] - 17s 729us/step - loss: 4.0347e-05
Epoch 10/10
23383/23383 [==============================] - 17s 734us/step - loss: 4.0280e-05


In [ ]:
x_test

,id,cluster,day,asset,md,bc,ret_1,ret_2,ret_3,ret_4,...,ret_14,ret_15,ret_16,ret_17,ret_18,ret_19,ret_20,ret_21,ret_22,ret_23
0,491038.0,1464.0,13.0,7.0,-5.422238,0.005959,0.033248,-0.011551,0.011686,-0.009901,...,0.044199,0.022487,-0.048512,0.021074,-0.007989,0.018121,0.009888,-0.033290,0.022957,0.017162
1,491039.0,1464.0,15.0,3.0,-4.944505,0.004303,0.003802,-0.016856,-0.000193,0.001927,...,0.010023,0.005725,-0.009108,-0.003638,0.001345,0.046065,-0.040367,0.001912,0.001912,-0.013168
2,491040.0,1464.0,14.0,3.0,-4.944505,0.004303,-0.009852,0.017987,-0.006579,0.013245,...,-0.013133,0.011407,0.007519,0.009328,-0.016636,-0.013158,-0.001905,-0.003817,0.005747,0.011429
3,491041.0,1464.0,13.0,3.0,-4.944505,0.004303,0.003891,0.001938,0.015474,-0.005714,...,-0.006106,-0.026744,0.010954,0.021120,-0.011331,-0.028743,0.025098,-0.046592,-0.015332,0.010705
4,491042.0,1464.0,12.0,3.0,-4.944505,0.004303,0.024691,0.028112,0.003906,-0.021401,...,0.004536,0.056352,-0.033457,-0.007308,0.007361,0.001923,-0.007678,-0.023211,0.030891,-0.024203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217902,708940.0,2090.0,17.0,3.0,-7.335009,0.658364,-0.008482,0.008104,0.001340,-0.000446,...,-0.015578,-0.008352,0.000887,-0.019486,-0.023487,-0.017114,0.011765,-0.020465,-0.009497,0.012464
217903,708941.0,2090.0,18.0,3.0,-7.335009,0.658364,-0.000964,0.016884,-0.015655,-0.009639,...,-0.011759,0.025781,-0.015950,0.012279,-0.003882,-0.007793,0.006382,0.007317,0.009201,0.011036
217904,708942.0,2090.0,19.0,3.0,-7.335009,0.658364,0.009259,0.008691,-0.002393,-0.007198,...,0.003498,-0.005478,0.018528,-0.002950,-0.004931,0.008424,-0.005405,0.011364,0.001466,-0.012195
217905,708943.0,2090.0,0.0,4.0,-7.394441,0.642147,0.004223,0.005887,-0.005853,-0.000841,...,-0.004227,0.001698,0.004237,0.005063,-0.004198,-0.007589,0.002549,-0.002542,-0.011045,0.024055


In [ ]:
x_t = x_test.drop(columns=['id','asset','day','cluster'])

In [ ]:
y_test = model.predict(x_t)
y_test = pd.DataFrame(y_test)
y_test = 2 * y_test - 1
y_test['sample_id'] = x_test['cluster'] * 21 + x_test['day']
y_test.rename(columns={0: 'target'}, inplace=True)
y_test = y_test.groupby('sample_id').mean()

In [ ]:
y_test

,target
sample_id,
30744.0,-0.001761
30745.0,-0.000817
30746.0,0.001689
30747.0,0.000927
30748.0,0.000100
...,...
43906.0,0.000406
43907.0,0.001706
43908.0,0.000414


In [ ]:
y_test.to_csv('y_test_DL6.csv')

# Essai 3: 

In [10]:
x_train_rnn = x_train.copy()
y_train_rnn = y_train.copy()

In [ ]:
y_train_rnn

,target
0,0.499228
1,0.489139
2,0.509788
3,0.503297
4,0.483645
...,...
908953,0.499730
908954,0.503535
908955,0.495447
908956,0.498020


In [12]:
def rnn_transform(cluster: int, day: int):
    df_rnn = x_train_rnn.where(x_train['cluster']==cluster).where(x_train['day']==day).dropna()
    df_rnn_y = df_rnn['target'].to_frame()
    df_rnn.drop(columns=['sample_id','target','id','cluster','asset','day'], inplace=True)
    if df_rnn.empty:
        return None, None
    df_rnn = torch.tensor(df_rnn.values.astype(np.float32))
    df_rnn_y = torch.tensor(df_rnn_y.values.astype(np.float32))
    return torch.stack([df_rnn]), torch.stack([df_rnn_y])

In [ ]:
rnn_transform(0,1)[1].shape

torch.Size([342, 1])

In [ ]:
torch.stack([train_x]).shape

torch.Size([1, 342, 25])

In [13]:
epochs = 10
inputSize = 25
hiddenSize = 50
numLayers = 1
lr = 1e-3
loss = torch.nn.MSELoss()
nn = torch.nn.RNN(inputSize, hiddenSize, numLayers, nonlinearity='relu', batch_first=True)
nn2 = torch.nn.Linear(hiddenSize, 1)
optimizer = torch.optim.Adam(list(nn.parameters()) + list(nn2.parameters()))
for day in range(21):
    for cluster in range(20):
        X, Y = rnn_transform(cluster, day)
        if X == None: continue
        for epoch in range(epochs):
            optimizer.zero_grad()
            output, _ = nn(X)
            res = nn2(output)
            loss_value = loss(res, Y)
            loss_value.backward()
            optimizer.step()
        if cluster % 5 == 0:
            print(f"Day: {day}, Cluster: {cluster}, Loss: {loss_value.item()}")

Day: 0, Cluster: 0, Loss: 0.0004048298578709364
Day: 0, Cluster: 5, Loss: 0.00020248083455953747
Day: 0, Cluster: 10, Loss: 0.00014798172924201936
Day: 0, Cluster: 15, Loss: 0.00014706305228173733
Day: 1, Cluster: 0, Loss: 0.00012638951011467725
Day: 1, Cluster: 5, Loss: 3.234167888876982e-05
Day: 1, Cluster: 10, Loss: 0.00013970684085506946
Day: 1, Cluster: 15, Loss: 9.086988575290889e-05
Day: 2, Cluster: 0, Loss: 6.766342994524166e-05
Day: 2, Cluster: 5, Loss: 0.00034142780350521207
Day: 2, Cluster: 10, Loss: 0.00010306390322512016
Day: 2, Cluster: 15, Loss: 0.00019584577239584178


KeyboardInterrupt: 

In [ ]:
torch.zeros(rnn_transform(0,0)[0].shape).shape

torch.Size([1, 342, 25])